In [1]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

embedding_model = SentenceTransformer(EMBEDDING_MODEL)

C:\Users\tarun\PyCharmProjects\lara\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Parquets file load

In [4]:
parquet_file = "rag_data\\raw_data\\0000.parquet"
output_file = "rag_data\\raw_data\\0000.json"

In [5]:
df = pd.read_parquet(parquet_file)

In [6]:
df.head()

,text
0,Venus
1,==============================================
2,
3,This section aims to cover the *functions* tha...
4,


In [7]:
#### PROCESS DATA HERE ####
document = df['text'].tolist()

embeddings = embedding_model.encode(document, convert_to_tensor=False).tolist()

In [8]:
try:
    with open(output_file,'r',encoding='utf-8') as f:
        existing_index = json.load(f)
        if "documents" not in existing_index:
            existing_index["documents"] = []
        if "embeddings" not in existing_index:
            existing_index["embeddings"] = []
except FileNotFoundError:
    existing_index = {"documents":[],"embeddings":[]}

In [9]:
#### APPEND DOCS ETC ####

existing_index["documents"].extend(document)
existing_index["embeddings"].extend(embeddings)

In [11]:
with open(output_file,'w',encoding='utf-8') as f:
    json.dump(existing_index,f)

print(f"Index updated with len")

Index updated with len


In [16]:
def concat_files(file_1, file_2, file_3):
    file_1 = json.load(open(file_1))
    file_2 = json.load(open(file_2))
    file_1["documents"].extend(file_2["documents"])
    file_1["embeddings"].extend(file_2["embeddings"])
    with open(file_3,'w',encoding='utf-8') as f:
        json.dump(file_1,f)

In [17]:
concat_files("rag_data/raw_data/0000.json", "hamilton.json", "rag_data/raw_data/hamilton2.json")

In [1]:
import json

# Load the original JSON structure from a file or raw string
with open("raw_data\\venus_errors.json", "r") as file:
    data = json.load(file)

# Extract the error handling details
errors = data[0]['details']['error_handling']

# Loop through the errors and create separate JSON metadata blocks
output_errors = []
for error in errors:
    formatted_error = {
        "type": "error_handling",
        "metadata": {
            "title": f"Error Code {error['error_code']}: {error['error_title']}",
            "description": f"Details about the Venus error code {error['error_code']} ({error['error_title']}).",
            "tags": ["runtime_errors", "Venus", "error_handling", error['error_code']],
            "documentation_url": "https://venuslibrarydocumentation.readthedocs.io/en/latest/RuntimeErrors.html#runtime-errors"
        },
        "details": {
            "error_handling": [
                {
                    "error_code": error["error_code"],
                    "error_title": error["error_title"],
                    "description": error["description"],
                    "user_action": error.get("user_action", "No user action provided."),
                    "developer_action": error.get("developer_action", "No developer action provided.")
                }
            ]
        }
    }
    output_errors.append(formatted_error)

# Save each error to a separate JSON file or combine into a larger structure
for i, error_block in enumerate(output_errors):
    filename = f"error_{error_block['details']['error_handling'][0]['error_code']}.json"
    with open(filename, 'w') as output_file:
        json.dump(error_block, output_file, indent=4)

# Optionally save to a combined single file
with open("raw_data/reformatted_errors.json", "w") as combined_file:
    json.dump(output_errors, combined_file, indent=4)

print("Errors reformatted and saved successfully.")


Errors reformatted and saved successfully.
